# Brightness temperature of 21-cm line

This code will calculate brightness temperature of 21 cm line in hydrogen in a given simulation. To do this, it needs:
1. density distribution of neutral hydrogen ($n_{\text{H}}\ [\text{cm}^{-3}]$)
2. radial velocity distribution ($v_z\ [\text{km/s}]$ in the code)
3. temperature distribution ($T\ [\text{K}]$)
4. local standard of rest (LSR) velocity interval and desired resolution ($v_{\text{LSR}}\ [\text{km/s}]$)
    
The calculation is done with the following equation $\left[^1\right]$:

$$T_{\text{br}}(v_{\text{LSR}},x,y)=5.49\cdot 10^{-14}\ \frac{\text{K cm}^3}{\text{s}}\  \int n_{\text{H}}(x,y,z) \frac{exp\left[ -\frac{\left( v_{\text{LSR}}-v_z(x,y,z)\right)^2}{2 b^2(x,y,z)}\right]}{\sqrt{2 \pi}\  b(x,y,z)} \ \text{d}z$$

where  $$b(x,y,z)=\sqrt{\frac{kT(x,y,z)}{m}}$$

$m$ is hydrogen atom mass, $k$ is Boltzmann constant and $z$ is the line of sight axis (in agreement with taking $v_z$ as radial velocity). Besides this, there is an option in the code for including the telescope beam effects and the spectral resolution where the beam is considered to be Gaussian. This means that the code will compute the following $\left[^2\right]$:

$$T_{\text{br}}^{\text{meas}}=B(x,y)*T_{\text{br}}(v_{\text{LSR}},x,y)$$

where $b$ from the upper equation will now be $\left[^2\right]$:

$$b'(x,y,z)=\sqrt{\frac{kT(x,y,z)}{m}+\sigma_{\text{psf}}^2}$$

where $\sigma_{\text{psf}}$ is standard deviation of the point spread function connected to the spectral resolution. $B(x,y)$ is the spatial Gaussian beam which is convolved ($*$ sign) with the brightness temperature from the upper equation.

The equation for brightness temperature is valid for an optically thin medium so the code will give meaningful results only if this approximation is satisfied. You have an option to check this in the last few cells of the code where column density is derived from both, brightness temperature using equation $\left[^1\right]$ 

$$N_{\text{H}}=1.823\cdot 10^{13}\ \frac{\text{s}}{\text{K cm}^3} \int T_{\text{br}} \ \text{d}v_z$$

and direct integration of the density using 

$$N_{\text{H}}'=\int n_{\text{H}} \ \text{d}z $$

These two approaches are then compared.

The code below contains cells which have a description paragraph above them.

As it is already mentioned, the line of sight axis is denoted with $z$ in the code and you will be able to choose the axis from your cube as the line of sight axis ($z$).

<br>

$\left[^1\right]$ Spitzer, L., Physical Processes in the Interstellar Medium, Chapter 3.3 b.

$\left[^2\right]$ Miville-Deschênes, M.-A., Martin, P. G., 2007, A&A 469, 189–199

In [ ]:
from astropy.io import fits
import numpy as np
import scipy.integrate
from scipy import signal
import pylab as plt
%matplotlib inline

###### Data collecting

In this cell you need to give some inputs mentioned before. Every line which requires change is noted with 'input'. Also pay attention to units given right after the code line. 

In [ ]:
cube_data1=fits.open('cube_nH.fits')    #input: fits files which 
cube_data2=fits.open('cube_vz.fits')    # contain needed quantities
cube_data3=fits.open('cube_T.fits')

nH=cube_data1[0].data #cm^(-3)  [hydrogen density]  -input: extract the right data from the cubes.
vz=cube_data2[0].data #km/s     [radial velocity]   -input
T=cube_data3[0].data #K         [temperature]       -input

nH_hdr=cube_data1[0].header    #input: extract the appropriate headers.
vz_hdr=cube_data2[0].header    #input
T_hdr=cube_data3[0].header     #input

beam='True'                                    #input: 'True' if you want to include the telescope beam in the measurements
beam_FWHM=3                                    #input: resolution in pixel space
beam_std=beam_FWHM/(2*np.sqrt(2*np.log(2)))

v_lsr_min=-50 #km/s    -input: minimum value of LSR velocity
v_lsr_max=50 #km/s     -input: maximum value of LSR velocity
v_lsr_npix=209 #km/s   -input: number of pixels in LSR velocity space

vz_sp_res='True'                                            #input: 'True' if you want to include the spectral resolution
vz_sp_res_FWHM=1.44 #km/s                                   -input: spectral resolution
vz_sp_res_std=vz_sp_res_FWHM*10**5/(2*np.sqrt(2*np.log(2))) #cm/s

verification='False'    #input: 'True' if you want to check the optically thin medium approximation

case=2    #input: case=0,1,2 defines the line of sight axis from your header (z is the line of sight axis)

if case==2:                        
    x_crval=nH_hdr['CRVAL0']    
    y_crval=nH_hdr['CRVAL1']    
    z_crval=nH_hdr['CRVAL2']    

    x_crpix=nH_hdr['CRPIX0']    
    y_crpix=nH_hdr['CRPIX1']    
    z_crpix=nH_hdr['CRPIX2']    
    
    x_npix=nH_hdr['NAXIS3']    
    y_npix=nH_hdr['NAXIS2'] 
    z_npix=nH_hdr['NAXIS1'] 

    x_del=nH_hdr['CDELT0']    
    y_del=nH_hdr['CDELT1']    
    z_del=nH_hdr['CDELT2']
    
    nH=nH_d
    vz=vz_d
    T=T_d
    
elif case==1:
    x_crval=nH_hdr['CRVAL2']    
    y_crval=nH_hdr['CRVAL0']    
    z_crval=nH_hdr['CRVAL1']    

    x_crpix=nH_hdr['CRPIX2']    
    y_crpix=nH_hdr['CRPIX0']    
    z_crpix=nH_hdr['CRPIX1']    
    
    x_npix=nH_hdr['NAXIS1']    
    y_npix=nH_hdr['NAXIS3']    
    z_npix=nH_hdr['NAXIS2']    

    x_del=nH_hdr['CDELT2']    
    y_del=nH_hdr['CDELT0']    
    z_del=nH_hdr['CDELT1']    
    
    nH=np.moveaxis(nH_d, [0,1,2], [1,2,0])
    vz=np.moveaxis(vz_d, [0,1,2], [1,2,0])
    T=np.moveaxis(T_d, [0,1,2], [1,2,0])
    
elif case==0:                            
    x_crval=nH_hdr['CRVAL1']    
    y_crval=nH_hdr['CRVAL2']    
    z_crval=nH_hdr['CRVAL0']    

    x_crpix=nH_hdr['CRPIX1']    
    y_crpix=nH_hdr['CRPIX2']    
    z_crpix=nH_hdr['CRPIX0']    
    
    x_npix=nH_hdr['NAXIS2']    
    y_npix=nH_hdr['NAXIS1']    
    z_npix=nH_hdr['NAXIS3']    

    x_del=nH_hdr['CDELT1']    
    y_del=nH_hdr['CDELT2']    
    z_del=nH_hdr['CDELT0']    
    
    nH=np.moveaxis(nH_d, [0,1,2], [2,0,1])
    vz=np.moveaxis(vz_d, [0,1,2], [2,0,1])
    T=np.moveaxis(T_d, [0,1,2], [2,0,1])

x_size=x_npix*x_del #pc
y_size=y_npix*y_del #pc
z_size=z_npix*z_del #pc

###### Making the numpy arrays for storing the output

Here we make a $v_{\rm LSR}$ interval and two numpy arrays. Intfun will be for storing the integrand, Tb will be the brightness temperature before convolution with the beam, and Tb_meas will be the measured brightness temperature at the end.

In [ ]:
v_lsr=np.linspace(v_lsr_min, v_lsr_max, v_lsr_npix)

Intfun=np.zeros((x_npix, y_npix, z_npix, len(v_lsr)), dtype=float)
Tb=np.zeros((x_npix, y_npix, len(v_lsr)), dtype=float)
Tb_meas=np.zeros((x_npix, y_npix, len(v_lsr)), dtype=float)

###### Calculation of the integrand

Here we explicitly calculate the integrand using given data from the cube. To do this we choose the $cm$ unit for length and change everything appropriately.

In [ ]:
m=1.6736e-27 #kg    -Hydrogen mass
k=1.38e-19 #cm^2 kg s^(-2) K^(-1)     -Boltzmann constant

if vz_sp_res!='True':
    vz_sp_res_std=0
    
b=np.sqrt((k/m)*T+vz_sp_res_std**2)

vz_cm=vz*1e+5 #cm/s
v_lsr_cm=v_lsr*1e+5 #cm/s

for l in range(len(v_lsr)):
    Intfun[:,:,:,l]=5.49e-14*nH*(1/(np.sqrt(2*np.pi)*b))*np.exp(-(v_lsr_cm[l]-vz_cm)**2/(2*b**2))

###### Integration

This cell computes the integration to get the brightness temperature. Line of sight axis unit is also changed to $cm$ to be consistent with the integrand. 

In [ ]:
z_pc=np.linspace(0, z_size-z_del, z_npix) #pc
z_cm=z_pc*3.086e+18 #cm

Tb=scipy.integrate.simps(Intfun, z_cm, axis=2)

###### Obtaining the measurements

This cell computes the convolution of the Gauss function and the Tb spectra computed before.

In [ ]:
if beam=='True':
    for i in range(len(v_lsr)):
        Tb_meas[:,:,i] = gaussian_filter(Tb[:,:,i], sigma=beam_std)
else:
    Tb_meas=Tb

###### Plotting the Tb spectra

Here we plot the given Tb spectra for desired pixel. You can choose the pixel position in the lines marked with 'changeable' (make sure this numbers are not greater than x_npix and y_npix), or if nothing is changed the cell will plot a spectra at the center of the picture.

In [ ]:
x_pix=int(x_npix/2)    #changeable
y_pix=int(y_npix/2)    #changeable

plt.figure(figsize=(10,6)) 
plt.plot(v_lsr, Tb_meas[x_pix,y_pix,:], color='blue')
plt.xlabel('$v_{LSR}\ [km/s]$')
plt.ylabel('$T_b\ [K]$')
plt.show()

###### Plotting the spatial dependence of Tb

To plot the spatial dependence of Tb you need to choose one LSR velocity. The line for this is marked with 'changeable'. If you don't change this, the cell wil plot for the LSR velocity in the middle of your interval.

In [ ]:
v_lsr_pix=int(len(v_lsr)/2)    #changeable

plt.figure(figsize=(10,6))
plt.imshow(np.moveaxis(Tb_meas[:,:,v_lsr_pix], [0,1], [1,0]), origin='lower', extent=(x_crval-x_crpix*x_del, 
                    x_crval+(x_npix-x_crpix)*x_del, y_crval-y_crpix*y_del, y_crval+(y_npix-y_crpix)*y_del))
plt.xlabel('$x\ [pc]$')
plt.ylabel('$y\ [pc]$')
colb=plt.colorbar()
colb.set_label('$T_b\ [K]$')

###### Saving the output

This cell saves the calculated brightness temperature as fits file in the desired directory which you need to specify. You do this in the line marked with 'input'.

In [ ]:
hdr=fits.Header()

if case==2:
    hdr['CRVAL0']=nH_hdr['CRVAL0']
    hdr['CRPIX0']=nH_hdr['CRPIX0']
    hdr['CDELT0']=nH_hdr['CDELT0']
    hdr['CTYPE0']=nH_hdr['CTYPE0']
    hdr['CUNIT0']=nH_hdr['CUNIT0']

    hdr['CRVAL1']=nH_hdr['CRVAL1']
    hdr['CRPIX1']=nH_hdr['CRPIX1']
    hdr['CDELT1']=nH_hdr['CDELT1']
    hdr['CTYPE1']=nH_hdr['CTYPE1']
    hdr['CUNIT1']=nH_hdr['CUNIT1']
    
elif case==1:
    hdr['CRVAL0']=nH_hdr['CRVAL2']
    hdr['CRPIX0']=nH_hdr['CRPIX2']
    hdr['CDELT0']=nH_hdr['CDELT2']
    hdr['CTYPE0']=nH_hdr['CTYPE2']
    hdr['CUNIT0']=nH_hdr['CUNIT2']

    hdr['CRVAL1']=nH_hdr['CRVAL0']
    hdr['CRPIX1']=nH_hdr['CRPIX0']
    hdr['CDELT1']=nH_hdr['CDELT0']
    hdr['CTYPE1']=nH_hdr['CTYPE0']
    hdr['CUNIT1']=nH_hdr['CUNIT0']
    
elif case==0:
    hdr['CRVAL0']=nH_hdr['CRVAL1']
    hdr['CRPIX0']=nH_hdr['CRPIX1']
    hdr['CDELT0']=nH_hdr['CDELT1']
    hdr['CTYPE0']=nH_hdr['CTYPE1']
    hdr['CUNIT0']=nH_hdr['CUNIT1']

    hdr['CRVAL1']=nH_hdr['CRVAL2']
    hdr['CRPIX1']=nH_hdr['CRPIX2']
    hdr['CDELT1']=nH_hdr['CDELT2']
    hdr['CTYPE1']=nH_hdr['CTYPE2']
    hdr['CUNIT1']=nH_hdr['CUNIT2']

hdr['CRVAL2']=(v_lsr_min+v_lsr_max)/2
hdr['CRPIX2']=int(np.floor(len(v_lsr)/2))
hdr['CDELT2']=v_lsr[1]-v_lsr[0]
hdr['CTYPE2']='V_z'
hdr['CUNIT2']='km/s'

hdu=fits.PrimaryHDU(Tb_meas,hdr)
hdul=fits.HDUList([hdu])
hdul.writeto('Brightness_temperature.fits') #input

## Verification of the optically thin medium approximation

###### Making the numpy arrays for storing the output

This cell makes the numpy arrays for saving the column densities from the two approaches described in the introduction.

In [ ]:
if verification=='True':
    cd_tb=np.zeros((x_npix,y_npix), dtype=float)
    cd=np.zeros((x_npix,y_npix), dtype=float)

###### Column density from brightness temperature

In this cell the integration of the brightness temperature over the velocity axis is computed in order to get the column density.

In [ ]:
if verification=='True':
    intfun=1.823e+13*Tb
    cd_tb=scipy.integrate.simps(intfun, v_lsr_cm, axis=2)

###### Column density from density

Cell computes the integration of density along the $z$-axis to obtain the appropriate column density.

In [ ]:
if verification=='True':
    cd=scipy.integrate.simps(nH, z_cm, axis=2)

###### Plotting the column densities from the two approaches

This cell is plotting the column density from density over the column density from the brightness temperature and also the column density from density over the column density from density to see if there is a difference between these two quantities. If the plots are the same then your data is optically thin and the calculation of the brightness temperature is valid.

In [ ]:
if verification=='True':
    plt.plot(cd.flatten(), cd_tb.flatten(), marker='.', color='blue', label='$N_H\ [cm^{-2}]$')
    plt.plot(cd.flatten(), cd.flatten(), color='orange', label='$N_H\'\ [cm^{-2}]$')
    plt.xlabel('$N_H\'\ [cm^{-2}]$')
    plt.legend()